In [ ]:
# 환경변수 설정
from dotenv import load_dotenv
load_dotenv()

### 요약봇 (Agent)

- 회의록 요약 기능과 이메일 요약 기능을 함수로 작성 > Tools로 생성
    - 서로 다른 프롬프트로 LLM 질의 > 응답 반환
    - `langchain.tools 패키지의 tool 데코레이터 활용 또는 Tool 활용
    - https://docs.langchain.com/oss/python/langchain/tools
- agent를 만들어 요청을 처리할 수 있도록 할 것

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

In [ ]:
test_meeting_note = """
주식회사 앨빈 - '슈퍼밴드' 10주년 기념 컴백 회의

2025년 3월 11일 (화) 14:00 - 16:00, 회사 회의실 A

참석자: 김앨빈 대표, 박다람 매니저, 이소리 홍보팀장, 최빛나 마케팅팀장, 강현우 프로듀서, 슈퍼밴드 멤버 전원

회의 시작. 김앨빈 대표 "이제 10주년인데 뭔가 크게 가야 하지 않겠어?" 다들 동의

일정 조율
- 발매일 6월 15일 확정. 멤버들 일정 문제 없음
- 티저 5월 1일부터 하나씩 공개. SNS 활용
- 뮤비 촬영 4월 10~12일. 로케이션 검토 중
- 쇼케이스 6월 14일로 가자. 장소는 아직 미정, 빠른 섭외 필요
- 월드투어? 일단 논의만, 일정 잡긴 어려움

앨범 컨셉 논의
- 앨범명 아이디어: Reignite, Revival, Firestorm (Reignite로 가는 분위기)
- 컨셉: 10년을 돌아보고 다시 불태우는 느낌. 레트로+현대 락 스타일로 가자
- 타이틀곡 가제: 불꽃처럼. 강렬한 락 사운드, 희망적인 가사
- 수록곡 10곡 예정. 기존 곡 리메이크도 넣을지 고민

마케팅 전략
- SNS 티저 영상, 멤버 개인 브이로그 촬영 진행
- 틱톡 챌린지 같은 거 해야 하나? 다들 긍정적 반응
- 팬미팅, 사인회 6월 중에. 세부 일정은 조율 필요
- 미디어 인터뷰 잡아야 함. 음악방송도 적극 참여
- 패션 브랜드 콜라보 제안 있음. 검토 필요

기타 논의
- 굿즈 제작? 팬클럽 한정 프리오더 진행 고려
- 다큐멘터리 제작할까? 가능성 있음, 제작팀 컨택 예정
- 멤버들 건강 스케줄 조정 신경 써야 함

결론
- 3월 말까지 세부 일정 확정
- 다음 회의 3월 25일 14:00

회의 종료 16:00
"""

test_email = """
제목: [마케팅팀 회의 공지] '슈퍼밴드' 10주년 컴백 마케팅 전략 논의  

팀원 여러분,  

안녕하세요. 최빛나 팀장입니다.
오늘 진행된 '슈퍼밴드' 10주년 컴백 회의 내용을 공유드립니다.  

컴백 일정이 6월 15일로 확정되었고, 이에 맞춰 마케팅 전략을 구체화해야 합니다. 주요 논의 사항은 다음과 같습니다.  

- **SNS 캠페인**: 티저 영상 및 멤버별 브이로그 콘텐츠 기획  
- **틱톡 챌린지**: 팬 참여 유도 방안 논의  
- **오프라인 이벤트**: 팬미팅, 사인회 일정 및 진행 방식 검토  
- **미디어 홍보**: 음악방송 및 인터뷰 스케줄 조율  
- **콜라보레이션**: 패션 브랜드 협업 가능성 검토  

이에 따라 **마케팅팀 내부 회의**를 진행하려 합니다.  
📅 일정: 3월 13일(수) 오후 2시  
📍 장소: 마케팅팀 회의실  

각자 아이디어를 정리해 오시면 원활한 논의가 가능할 것 같습니다.  
질문이나 제안이 있으면 미리 공유해주세요.  

감사합니다.  
최빛나 드림
"""

In [ ]:
# LLM
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)

# 회의록 요약 기능 (함수)
def summary_note_tool(text):

    prompt = f"""
다음 조건에 맞게 회의 내용을 요약해줘.

<조건>
1. 회의 장소, 일자, 참여자는 각각 설정할 것
2. 회의 내용을 대주제, 소주제로 나누어 작성할 것
3. 요약한 회의 내용을 불렛 형식으로 작성할 것

<요약할 이메일 내용>
{text}
"""
    return model.invoke(prompt)

# 이메일 요약 기능 (함수)
def summary_email_tool(text):

    prompt ="""
다음 조건에 맞게 이메일 내용을 요약해줘.

<조건>
1. 이메일 발신자를 따로 표시할 것
2. 메일 내용을 간략하게 요약해서 성할 것
3. 요청 사항이 있으면 불렛 형식으로 작성할 것

<요약할 이메일 내용>
{text}
"""
    return model.invoke(prompt)

In [ ]:
# Tool 생성 > Agent 생성 > 테스트 데이터로 확인
tools =[Tool(
    name="회의 요약",
    func=summary_note_tool,
    description="긴 회의록 내용을 요약한다."
    ),
    Tool(
    name="메일 요약",
    func=summary_email_tool,
    description="업무 이메일 내용을 요약한다."
    )
]

agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
result = agent.run("회의 내용 요약 : " + test_meeting_note)